In [1]:
# Michel Bierlaire
# Sun Oct 21 23:39:14 2018

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
from biogeme.expressions import Beta, DefineVariable

df = pd.read_csv("optimaDataset.dat",'\t')
database = db.Database("optimaDataset",df)
pd.options.display.float_format = '{:.3g}'.format
globals().update(database.variables)

### The condition CostCarCHF was not originally there.
exclude =  ((Choice == -1) + (  CostCarCHF   <  0  )) >0
database.remove(exclude)

#Parameters to be estimated
# Arguments:
#   1  Name for report. Typically, the same as the variable
#   2  Starting value
#   3  Lower bound
#   4  Upper bound
#   5  0: estimate the parameter, 1: keep it fixed
ASC_CAR	 = Beta('ASC_CAR',0,None,None,0)
ASC_SM	 = Beta('ASC_SM',0,None,None,0)
ASC_PT	 = Beta('ASC_PT',0,None,None,1)
BETA_COST	 = Beta('BETA_COST',0,None,None,0)
BETA_DIST_SM	 = Beta('BETA_DIST_SM',0,None,None,0)
BETA_DIST_CAR	 = Beta('BETA_DIST_CAR',0,None,None,0)
BETA_LANGUAGE_PT	 = Beta('BETA_LANGUAGE_PT',0,None,None,1)
BETA_TIME_CAR	 = Beta('BETA_TIME_CAR',0,None,None,0)
BETA_TIME_PT	 = Beta('BETA_TIME_PT',0,None,None,0)
BETA_TIME_CAR_FRENCH	 = Beta('BETA_TIME_CAR_FRENCH',0,None,None,0)
BETA_TIME_PT_FRENCH	 = Beta('BETA_TIME_PT_FRENCH',0,None,None,0)

# Define here arithmetic expressions for name that are not directly 
# available from the data

FrenchRegion  = DefineVariable('FrenchRegion', LangCode   ==  1 ,database)

#Utilities
CAR = ASC_CAR + BETA_TIME_CAR * TimeCar * (1-FrenchRegion) + BETA_TIME_CAR_FRENCH * TimeCar * FrenchRegion + BETA_COST * CostCarCHF + BETA_DIST_CAR * distance_km
SM = ASC_SM + BETA_DIST_SM * distance_km
PT = ASC_PT + BETA_TIME_PT * TimePT * (1-FrenchRegion)  + BETA_TIME_PT_FRENCH * TimePT * FrenchRegion + BETA_COST * MarginalCostPT
V = {1: CAR,2: SM,0: PT}
av = {1: 1,2: 1,0: 1}

# Logit model, with availability conditions
logprob = models.loglogit(V,av,Choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_socioecon_optima"
results = biogeme.estimate()
# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)
print(f"Nbr of observations: {database.getNumberOfObservations()}")
print(f"LL(0) =    {results.data.initLogLike:.3f}")
print(f"LL(beta) = {results.data.logLike:.3f}")
print(f"rho bar square = {results.data.rhoBarSquare:.3g}")
print(f"Output file: {results.data.htmlFileName}")


                       Value  Std err  t-test  p-value  Rob. Std err  \
ASC_CAR                 0.61    0.102    5.99 2.06e-09         0.109   
ASC_SM                0.0984    0.175   0.562    0.574         0.308   
BETA_COST            -0.0651  0.00745   -8.74        0         0.012   
BETA_DIST_CAR        0.00839   0.0038     2.2   0.0275        0.0057   
BETA_DIST_SM          -0.239   0.0205   -11.6        0        0.0539   
BETA_TIME_CAR        -0.0394  0.00604   -6.53 6.54e-11        0.0117   
BETA_TIME_CAR_FRENCH -0.0533    0.008   -6.66 2.72e-11       0.00996   
BETA_TIME_PT         -0.0109  0.00168   -6.47 9.69e-11       0.00332   
BETA_TIME_PT_FRENCH   -0.024   0.0033   -7.26  3.9e-13       0.00325   

                      Rob. t-test  Rob. p-value  
ASC_CAR                      5.58      2.39e-08  
ASC_SM                      0.319         0.749  
BETA_COST                   -5.43      5.56e-08  
BETA_DIST_CAR                1.47         0.141  
BETA_DIST_SM                -